In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs, make_circles
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import KMeans, AgglomerativeClustering
import numpy as np
from silhouette import make_figure, make_silhouette

In [ ]:
n_clusters = 5
data, cluster_labels = make_blobs(n_samples=500, n_features=2000, centers=n_clusters, center_box=(-10.0, 10.0), cluster_std=[1,1,4,1,1], random_state=42)
data = data + np.abs(np.min(data))

In [ ]:
make_figure(data, cluster_labels)

In [ ]:
metrics = ['euclidean', 'correlation', 'cosine']

In [ ]:
make_silhouette(cluster_labels)

In [ ]:
model = KMeans(n_clusters=n_clusters, n_init=1000)

In [ ]:
out = model.fit_predict(data)

In [ ]:
make_figure(out)

In [ ]:
make_silhouette(out)

In [ ]:
model=AgglomerativeClustering(n_clusters=5, linkage='average')

In [ ]:
hierout = model.fit_predict(data)

In [ ]:
make_figure(hierout)

In [ ]:
make_silhouette(hierout)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
lda = LatentDirichletAllocation(n_components=10, random_state=42)

In [ ]:
ldaout = np.argmax(lda.fit_transform(data), axis=1)

In [ ]:
make_figure(ldaout)

In [ ]:
make_silhouette(ldaout)

# Null model from data

In [ ]:
import pandas as pd
import cloudpickle as pickle

In [ ]:
df = pd.read_csv("../topics/datasets/gtex10/mainTable.csv", index_col=0)
df_files = pd.read_csv("../topics/datasets/gtex10/files.dat").set_index("SAMPID")

In [ ]:
df_tissue = df.transpose().applymap(lambda tpm: np.log2(tpm+1)).join(df_files.loc[:,"SMTS"]).groupby("SMTS").median()
means = df_tissue.mean(1)
stds = df_tissue.std(1)

In [ ]:
n_clusters = df_tissue.shape[0]
data, cluster_labels = make_blobs(n_samples=df.shape[1], n_features=df_tissue.shape[1], centers=df_tissue.values, cluster_std=stds, random_state=42)

In [ ]:
assert(data.T.shape==df.shape)

In [ ]:
with open("figs.pkl", "rb") as file:
    figs = pickle.load(file)

In [ ]:
def orange_color():
    while True:
        yield "orange"
        
orange_itarator = orange_color()

In [ ]:
make_silhouette(data,
                "SMTS+null", 
                df_tissue.index, 
                cluster_labels, 
                cluster_labels.max()+1, 
                metrics = ["euclidean", "cosine", "correlation"],
                color_iterator=orange_itarator, 
                figs=figs
               )

In [ ]:
figs[0]